In [ ]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df=pd.read_csv("/content/G2 software - CRM Category Product Overviews.csv")
df.columns


Index(['url', 'product_name', 'rating', 'description', 'product_url', 'seller',
       'ownership', 'seller_website', 'headquarters', 'total_revenue',
       'social_media_profiles', 'seller_description', 'reviews_count',
       'discussions_count', 'pros_list', 'cons_list', 'competitors',
       'highest_rated_features', 'lowest_rated_features', 'rating_split',
       'pricing', 'official_screenshots', 'official_downloads',
       'official_videos', 'categories', 'user_ratings', 'languages_supported',
       'year_founded', 'position_against_competitors', 'overview', 'claimed',
       'logo', 'reviews', 'top_alternatives', 'top_alternatives_url',
       'full_pricing_page', 'badge', 'what_is_description', 'main_category',
       'main_subject', 'Features', 'region', 'country_code',
       'software_product_id', 'overview_provided_by'],
      dtype='object')

In [ ]:
# Set options to display all columns
pd.set_option('display.max_columns', None)

# Display the DataFrame
df.head(2)


,url,product_name,rating,description,product_url,seller,ownership,seller_website,headquarters,total_revenue,social_media_profiles,seller_description,reviews_count,discussions_count,pros_list,cons_list,competitors,highest_rated_features,lowest_rated_features,rating_split,pricing,official_screenshots,official_downloads,official_videos,categories,user_ratings,languages_supported,year_founded,position_against_competitors,overview,claimed,logo,reviews,top_alternatives,top_alternatives_url,full_pricing_page,badge,what_is_description,main_category,main_subject,Features,region,country_code,software_product_id,overview_provided_by
0,https://www.g2.com/products/efficy-crm/reviews,Efficy CRM,4.5,The Efficy extendable CRM platform (xCRM) orga...,https://www.efficy.com/efficy-crm-features/,Efficy,NaN,https://www.efficy.com/,"Brussels, Belgium",NaN,"[{""properties"":[{""name"":""followers"",""value"":""1...",NaN,102.0,3.0,"[{""count"":""50"",""text"":""Ease of Use""},{""count"":...","[{""count"":""21"",""text"":""Learning Curve""},{""coun...","[{""comparsion_link"":""https://www.g2.com/compar...",NaN,NaN,"[{""rating"":""5 star"",""reviews_count"":""74""},{""ra...","[{""package"":""SMB"",""price"":""Starting at €49.00""...",NaN,NaN,NaN,"[""CRM"",""Email Marketing""]",NaN,"Arabic, Czech, Danish, German, Greek, English,...",2005.0,Product Description The Efficy extendable CRM ...,"At last, a CRM solution that truly adapts to y...",True,https://images.g2crowd.com/uploads/product/ima...,"[{""Author"":""Dhr***D."",""Author_Id"":""5499300"",""C...","[{""link"":""https://www.g2.comundefined""},{""link...",https://www.g2.com/products/efficy-crm/competi...,https://www.g2.com/products/efficy-crm/pricing,NaN,NaN,CRM Software,Home>CRM Software>Efficy CRM>Efficy CRM Reviews,"[{""Category"":""Platform"",""features"":[{""descript...",EU,BE,efficy-crm,Laetitia Baret
1,https://www.g2.com/products/salesboss/reviews,Salesboss,5.0,"SalesBoss is an all-in-one sales, marketing, c...",https://www.salesboss.ai/pricing,Salesboss,NaN,https://www.salesboss.ai/,NaN,NaN,NaN,NaN,1.0,NaN,"[{""count"":""1"",""text"":""Ease of Use""},{""count"":""...",NaN,NaN,NaN,NaN,"[{""rating"":""5 star"",""reviews_count"":""1""},{""rat...","[{""package"":""Business Plan"",""price"":""$99.00"",""...",NaN,NaN,NaN,"[""CRM""]",NaN,NaN,NaN,NaN,"SalesBoss is an all-in-one sales, marketing, c...",True,https://images.g2crowd.com/uploads/product/ima...,"[{""Author"":""******"",""Author_Id"":""3840237"",""Com...","[{""link"":""https://www.g2.com/products/salesfor...",https://www.g2.com/products/salesboss/competit...,https://www.g2.com/products/salesboss/pricing,NaN,NaN,CRM Software,Home>CRM Software>Salesboss>Salesboss Reviews,NaN,NaN,NaN,salesboss,Prit Pal


In [ ]:
# Count null values in the 'Features' column
null_count = df['Features'].isnull().sum()
# Print the result
print(f"Number of null values in 'Features' column: {null_count}")

Number of null values in 'Features' column: 48


In [ ]:
# Replace NaN in rating with 0
df['rating'] = df['rating'].fillna(0)

In [ ]:
# Count null values in the 'pros_list' column
null_count = df['pros_list'].isnull().sum()
# Print the result
print(f"Number of null values in 'pros_list' column: {null_count}")

Number of null values in 'pros_list' column: 30


In [ ]:
features_null_count = df['Features'].isnull().sum()
print(f"Number of rows where 'Features' is null: {features_null_count}")

pros_list_null_count = df['pros_list'].isnull().sum()
print(f"Number of rows where 'pros_list' is null: {pros_list_null_count}")

# Count where both are null
count = len(df[df['Features'].isnull() & df['pros_list'].isnull()])
print(f"Number of rows where both 'Features' and 'pros_list' are null: {count}")

# Count where Features is empty and pros_list is not
count1 = len(df[df['Features'].isnull() & ~df['pros_list'].isnull()])
print(f"Number of rows where 'Features' is empty and 'pros_list' is not empty: {count1}")

# Count where Features is not empty and pros_list is empty
count2 = len(df[~df['Features'].isnull() & df['pros_list'].isnull()])
print(f"Number of rows where 'Features' is not empty and 'pros_list' is empty: {count2}")

count3 = len(df[~df['Features'].isnull() & ~df['pros_list'].isnull()])
print(f"Number of rows where both 'Features' and 'pros_list' have values: {count3}")

# Count where either Features or pros_list (or both) has a value
count4 = len(df[~df['Features'].isnull() | ~df['pros_list'].isnull()])
print(f"Number of rows where either 'Features' or 'pros_list' (or both) have values: {count4}")

Number of rows where 'Features' is null: 48
Number of rows where 'pros_list' is null: 30
Number of rows where both 'Features' and 'pros_list' are null: 27
Number of rows where 'Features' is empty and 'pros_list' is not empty: 21
Number of rows where 'Features' is not empty and 'pros_list' is empty: 3
Number of rows where both 'Features' and 'pros_list' have values: 12
Number of rows where either 'Features' or 'pros_list' (or both) have values: 36


In [ ]:
def extract_feature_names(features_str):
    try:
        features_data = json.loads(features_str)
        all_feature_names = []
        for category_data in features_data:
            if 'features' in category_data and isinstance(category_data['features'], list):
                for feature in category_data['features']:
                    if 'name' in feature:
                        all_feature_names.append(f'"{feature["name"]}"')
        return all_feature_names
    except (json.JSONDecodeError, TypeError) as e:
        return []
def extract_pros_text(pros_str):
    try:
        pros_data = json.loads(pros_str)
        pros_text = []
        for item in pros_data:
            if isinstance(item, dict) and 'text' in item:
                # Enclose the text within double quotes
                pros_text.append(f'"{item["text"]}"')
        return pros_text
    except (json.JSONDecodeError, TypeError) as e:
        return []

# Create a copy of the DataFrame
df1 = df.copy()

# Replace NaN with empty JSON array in 'Features' column
df1['Features'] = df1['Features'].fillna('[]')

# Apply the function to extract feature names
df1['feature_names'] = df1['Features'].apply(extract_feature_names)

# Replace NaN with empty JSON array in 'pros_list' column
df1['pros_list'] = df1['pros_list'].fillna('[]')

# Apply the function to extract pros text
df1['pros_text'] = df1['pros_list'].apply(extract_pros_text)

# Create a new column 'combined_text'
df1['all_features'] = df1.apply(lambda row: row['pros_text'] if not row['feature_names'] else row['feature_names'], axis=1)

# Create a new DataFrame with selected columns
new_df = df1[['product_name', 'seller','main_category', 'categories', 'rating', 'all_features']].copy()
new_df = new_df.rename(columns={'product_name': 'software_name', 'seller': 'vendor'})
# Display the new DataFrame with the 'combined_text' column
new_df

,software_name,vendor,main_category,categories,rating,all_features
0,Efficy CRM,Efficy,CRM Software,"[""CRM"",""Email Marketing""]",4.5,"[""Customization"", ""Workflow Capability"", ""User..."
1,Salesboss,Salesboss,CRM Software,"[""CRM""]",5.0,"[""Ease of Use"", ""Time Efficiency""]"
2,Desktop Sales Office,The CRM Guide,CRM Software,"[""CRM""]",3.0,[]
3,Atendare,Inofly,CRM Software,"[""CRM"",""Email Marketing""]",5.0,"[""Customer Support"", ""Ease of Use"", ""Easy Impl..."
4,ClinchPad,ClinchPad Technologies Pvt Ltd,CRM Software,"[""CRM""]",4.8,[]
...,...,...,...,...,...,...
58,Smalution CRM,Smalution,CRM Software,"[""CRM""]",3.0,[]
59,Insale CRM,Zaw,CRM Software,"[""CRM""]",5.0,[]
60,eXert CRM,eXerterp,CRM Software,"[""CRM""]",4.0,[]
61,Directions CRM,Directions CRM,CRM Software,"[""CRM""]",0.0,[]


In [ ]:
new_df.head(20)

,software_name,vendor,main_category,categories,rating,all_features
0,Efficy CRM,Efficy,CRM Software,"[""CRM"",""Email Marketing""]",4.5,"[""Customization"", ""Workflow Capability"", ""User..."
1,Salesboss,Salesboss,CRM Software,"[""CRM""]",5.0,"[""Ease of Use"", ""Time Efficiency""]"
2,Desktop Sales Office,The CRM Guide,CRM Software,"[""CRM""]",3.0,[]
3,Atendare,Inofly,CRM Software,"[""CRM"",""Email Marketing""]",5.0,"[""Customer Support"", ""Ease of Use"", ""Easy Impl..."
4,ClinchPad,ClinchPad Technologies Pvt Ltd,CRM Software,"[""CRM""]",4.8,[]
5,Zimplu CRM,SE Telecom,CRM Software,"[""CRM""]",3.9,"[""Affordable"", ""Ease of Use"", ""Simplicity"", ""C..."
6,Zurmo,"Zurmo, Inc.",CRM Software,"[""CRM""]",4.6,"[""Customization"", ""Workflow Capability"", ""User..."
7,YetiForce CRM,YetiForce,CRM Software,"[""CRM""]",4.4,"[""Customization"", ""Workflow Capability"", ""User..."
8,webCRM,webCRM,CRM Software,"[""CRM"",""Email Marketing""]",4.3,[]
9,Webmecanik Pipeline,Webmecanik,CRM Software,"[""CRM""]",5.0,"[""Ease of Use"", ""Efficiency"", ""Simplicity""]"


In [ ]:
def get_relevant_vendors(query_keywords, df, threshold=0.5):
    """
    Extracts relevant vendors and their information based on query keywords,
    feature similarity, and sorts by similarity score.
    """
    # 1. Vendor Filtering (Case-Insensitive)
    query_keywords_lower = {keyword.lower() for keyword in query_keywords}
    relevant_vendors = df[df['all_features'].apply(lambda features: any(keyword in ' '.join(features).lower() for keyword in query_keywords_lower))].copy()

    # 2. Feature Similarity
    if relevant_vendors.empty:
        return pd.DataFrame(columns=['vendor', 'software_name', 'main_category','categories', 'all_features', 'similarity_score'])

    relevant_vendors = relevant_vendors.copy()
    relevant_vendors.loc[:, 'all_features_str'] = relevant_vendors['all_features'].apply(lambda x: ' '.join(x))

    # Create TF-IDF vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(relevant_vendors['all_features_str'])

    # Calculate cosine similarity with ALL vendors
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # Calculate average similarity for each vendor
    average_similarity = np.mean(similarity_matrix, axis=1)

    # Add average similarity scores to DataFrame
    relevant_vendors['similarity_score'] = average_similarity

    # 3. Threshold-Based Filtering and Sorting
    filtered_vendors = relevant_vendors[relevant_vendors['similarity_score'] >= threshold]
    filtered_vendors = filtered_vendors.sort_values(by=['similarity_score'], ascending=False)

    # Return DataFrame with desired columns, including similarity_score
    return filtered_vendors[['vendor','rating', 'software_name', 'main_category','categories', 'all_features', 'similarity_score']]


# Assuming you have your DataFrame 'new_df' loaded

# Get user input for query keywords
user_input = input("Enter query keywords separated by commas: ")
query_keywords = [keyword.strip() for keyword in user_input.split(",")]
if not any(query_keywords):
        print("No input keywords were provided. Please enter at least one keyword.")
else:
        # Call the function with user-provided keywords and threshold
        filtered_vendors_df = get_relevant_vendors(query_keywords, new_df, threshold=0.6)
        print(filtered_vendors_df)

Enter query keywords separated by commas: work
                           vendor  rating         software_name main_category  \
6                     Zurmo, Inc.     4.6                 Zurmo  CRM Software   
18  FinCRM Technologies Pvt. Ltd.     4.3                FinCRM  CRM Software   
23      Solid Performers Pvt. Ltd     4.9  Solid Performers CRM  CRM Software   
19                      Fireberry     4.8             Fireberry  CRM Software   
43                      QSOFT LLC     3.7                 Kommo  CRM Software   
56                   ProspectSoft     4.6          Prospect CRM  CRM Software   
7                       YetiForce     4.4         YetiForce CRM  CRM Software   
0                          Efficy     4.5            Efficy CRM  CRM Software   
36                     AllClients     4.6            AllClients  CRM Software   
33                      Insightly     4.2         Insightly CRM  CRM Software   
30                      Breakcold     4.7             Breakcol

In [ ]:
def rank_vendors(query_keywords, df, threshold=0.5, rating_weight=0.2):
    """
    Ranks vendors based on average feature similarity (weighted by individual feature match),
    overall vendor rating, and ensures search-selected vendors are included.

    Args:
        query_keywords (list): List of user-provided keywords.
        df (pd.DataFrame): DataFrame containing vendor information.
        threshold (float): Similarity threshold for initial filtering.
        rating_weight (float): Weight given to vendor rating in ranking.

    Returns:
        pd.DataFrame: Ranked vendors DataFrame with 'rank' column.

    """

    # 1. Get relevant vendors (using existing get_relevant_vendors function)
    relevant_vendors = get_relevant_vendors(query_keywords, df, threshold)

    # 2. Calculate weighted average feature similarity
    query_keywords_lower = {keyword.lower() for keyword in query_keywords}
    relevant_vendors['weighted_similarity'] = relevant_vendors['all_features'].apply(
        lambda features: sum(1 for keyword in query_keywords_lower if keyword in ' '.join(features).lower()) / len(query_keywords_lower)
    )
    relevant_vendors['weighted_similarity'] = relevant_vendors['weighted_similarity'] * (1 - rating_weight)

    # 3. Incorporate vendor rating (if available)
    if 'rating' in relevant_vendors.columns:
        relevant_vendors['rating_score'] = relevant_vendors['rating'] * rating_weight
        relevant_vendors['final_score'] = relevant_vendors['weighted_similarity'] + relevant_vendors['rating_score']
    else:
        relevant_vendors['final_score'] = relevant_vendors['weighted_similarity']

    # 4. Ensure search-selected vendors are included
    search_selected_vendors = relevant_vendors[relevant_vendors['similarity_score'] >= threshold]
    ranked_vendors = relevant_vendors.sort_values(by=['final_score'], ascending=False)

    # 5. Add rank column
    ranked_vendors['rank'] = ranked_vendors['final_score'].rank(ascending=False, method='first')

    if not any(query_keywords):  # Check for empty keywords
        # Sort by rating only, ignoring other factors
        ranked_vendors = df.sort_values(by=['rating'], ascending=False)
        ranked_vendors['rank'] = ranked_vendors['rating'].rank(ascending=False, method='first')

    return ranked_vendors

In [ ]:
user_input = input("Enter query keywords separated by commas: ")
query_keywords = [keyword.strip() for keyword in user_input.split(",")]
if not any(query_keywords):
  print("No input. Hence results are based on vendor rating alone")
ranked_vendors_df = rank_vendors(query_keywords, new_df, threshold=0.6, rating_weight=0.2)
ranked_vendors_df[['vendor','software_name','rating','all_features']]

Enter query keywords separated by commas: work


,vendor,software_name,rating,all_features
23,Solid Performers Pvt. Ltd,Solid Performers CRM,4.9,"[""Customization"", ""Workflow Capability"", ""User..."
19,Fireberry,Fireberry,4.8,"[""Customization"", ""Workflow Capability"", ""User..."
30,Breakcold,Breakcold,4.7,"[""Customization"", ""Workflow Capability"", ""User..."
56,ProspectSoft,Prospect CRM,4.6,"[""Customization"", ""Workflow Capability"", ""User..."
6,"Zurmo, Inc.",Zurmo,4.6,"[""Customization"", ""Workflow Capability"", ""User..."
53,EspoCRM Inc.,EspoCRM,4.6,"[""Customization"", ""Workflow Capability"", ""User..."
36,AllClients,AllClients,4.6,"[""Customization"", ""Workflow Capability"", ""User..."
25,Freshworks,Freshsales,4.5,"[""Customization"", ""Workflow Capability"", ""User..."
0,Efficy,Efficy CRM,4.5,"[""Customization"", ""Workflow Capability"", ""User..."
7,YetiForce,YetiForce CRM,4.4,"[""Customization"", ""Workflow Capability"", ""User..."
